<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 0 - Introducción a Redes Neuronales </H3>
<hr style="height:2px;border:none"/>

<H3 align='center'>Integrantes:</H3>
<H3 align='center'>Fernanda Weiss Rol: 201373536-3</H3>
<H3 align='center'>Ignacio Espinoza Rol: 201073527-3</H3>
<hr style="height:2px;border:none"/>

**Temas**  
* NNs por dentro: *back-propagation from scratch*.
* Principales hiperparámetros de *back propagation*
* Introducción a keras
* Verificación numérica de las derivadas implementadas.

** Formalidades **  
* Equipos de trabajo de: 2 personas (*cada uno debe estar en condiciones de responder preguntas sobre cada punto del trabajo realizado*)
* Se debe preparar un (breve) Jupyter/IPython notebook que explique la actividad realizada y las conclusiones del trabajo
* Fecha de entrega: 30 de Marzo.
* Formato de entrega: envı́o de link Github al correo electrónico del ayudante (*<francisco.mena.13@sansano.usm.cl>*) , incluyendo al profesor en copia (*<jnancu@inf.utfsm.cl>*). Por favor especificar el siguiente asunto: [Tarea0-INF395-I-2018]

<hr style="height:2px;border:none"/>


La tarea se divide en cuatro secciones:

[1.](#primero)   Back-propagation (BP) from *Scratch*   
[2.](#segundo)   Comparar back-propagation (BP) de Keras  
[3.](#tercero)   Verificación numérica del gradiente para una componente  
[4.](#cuarto)   Implementar momentum como variante

# Introducción

En el siguiente trabajo se implementará una red neuronal feed fordward con Back-propagation desde cero para resolver un problema de clasificación. Esta red se pondrá a prueba versus una implementación de una red neuronal proporcionada por [Keras](https://keras.io/), una API de alto nivel para redes neuronales escrita en Python.

Para los entrenamientos de cada modelo se utilizará el dataset *[Iris](https://es.wikipedia.org/wiki/Iris_flor_conjunto_de_datos)*, el cual posee datos de 50 muestras de cada una de tres especies de la planta Iris: Iris setosa, Iris virginica e Iris versicolor. Cada entrada del dataset presenta mediciones del ancho y largo se sus sépalos y pétalos, además del tipo de planta en cuestión (clase o etiqueta).

<img src="http://s5047.pcdn.co/wp-content/uploads/2015/04/iris_petal_sepal.png" alt="Mediciones de una planta" style="width: 300px;"/>

<a id="primero"></a>
## 1. Back-propagation (BP) from *Scratch*

BP (Back-propagation) es sin duda el paradigma dominante para entrenar redes neuronales *feed-forward*. En
redes grandes, diseñadas para problemas reales, implementar BP eficientemente puede ser una tarea delicada
que puede ser razonable delegar a una librerı́a especializada. Sin embargo, construir BP *from scratch* es muy
útil con fines pedagógicos.

$$ w^{(t+1)} \leftarrow w^{(t)} - \eta \nabla_{w^{(t)}} Loss $$
 
> a) Escriba un programa que permita entrenar una red FF con una arquitectura fija de 2 capa ocultas (con 32 neuronas en la primera capa y 16 en la segunda) y $K$ neuronas de salida, sin usar librerı́as, excepto eventualmente *numpy* para implementar operaciones básicas de algebra lineal. Por simplicidad, asuma funciones de activacion y error (*loss function*) diferenciables o subdiferenciables, además de tener la misma función de activación para las 2 capas ocultas. Adapte la arquitectura para un problema de clasificación de 3 clases, es decir la función de activación para la capa de salida debe ser **softmax** con número de neuronas $K$=3. Escriba funciones para:  
* (i)  implementar el *forward pass*  
* (ii) implementar el *backward pass*  
* (iii) implementar la rutina principal de entrenamiento, adoptando, por simplicidad, la variante cíclica aleatorizada de SGD (un ejemplo a la vez, pero iterando cíclicamente sobre una configuración aleatoria del conjunto de entrenamiento) con una tasa de aprendizaje fija de 0.1 y número de ciclos fijos (*epochs*).

> b) Escriba una función que permita hacer predicciones mediante la red FF definida anteriormente, sin usar librerı́as, excepto eventualmente *numpy*. Escriba una función vectorizada que implemente el forward pass sobre un conjunto de $n_{t}$ ejemplos, además de implementar la función de decisión, que a través de la salida de la red prediga el valor categórico de la clase (1, 2 o 3).

> c) Demuestre que sus programas funcionan en un problema de clasificación. Para esto utilice el dataset **iris**, disponible a través de la librería __[*sklearn*](http://scikit-learn.org)__, el cual corresponde a la clasificación de distintos tipos de plantas de iris (3 clases) mediante 4 características reales continuas específicas de la planta, deberá entrenar (ajustar) los pesos de la red para realizar la tarea encomendada, variando las funciones de error (*loss*) entre *categorical cross entropy* y *mean squared error*, además de variar las funciones de activación para las 2 capas ocultas entre  ReLU (Rectifier Linear Unit) y la función logística (*sigmoid*). Especifique explícitamente las funciones anteriores, así como sus gradientes. Recuerde que debe transformar las etiquetas usando *one hot vectors*.
<div class="alert alert-block alert-info">Es una buena práctica el normalizar los datos antes de trabajar con el modelo</div>
```python
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
X_train,y_train = load_iris(return_X_y=True)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
#transform target to one hot vector
import keras
y_onehot = keras.utils.to_categorical(y_train)
```
Para evaluar los resultados, construya un gráfico correspondiente al error de clasificación versus número
de epochs, utilizando sólo el conjunto de entrenamiento (el objetivo de esta sección es familiarizarse
con el algoritmo BP, no encontrar la mejor red). Grafique también la evolución de la función objetivo utilizada para el entrenamiento. Además de reportar el tiempo de entrenamiento mediante el algoritmo implementado.  
Por último, para alguna configuración elegida, reporte la matriz de confusión mediante el uso de librerías como *sklearn* o *keras*.

In [1]:
import numpy as np

In [4]:
# funciones de activación
epsilon =  1e-15

def sigmoid(x):
    if x > 0:
        x = np.maximum(epsilon, x) #si x es muy pequeño
    if x < 0:
        x = np.maximum(-600.,x)  #si x es muy negativo
    return 1. / (1. + np.exp(-x))
    
def gradient_sigmoid(x):
    return sigmoid(x) * (1. - sigmoid(x))

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

def gradient_softmax(x):
    return softmax(x) * (1 - softmax(x))

def relu(x):
    return np.maximum(x, 0)

def gradient_relu(x):
    return 1. * (x > 0)

def loss_function():
    pass

def gradient_loss_function():
    pass



def create_matrices(layers):
    # matrices de salidas de neuronas
    A = []
    A_derivate = [] 
    # matrices de errores
    error_outputs = []
    error_weights = []
    # pesos de la red
    network_weights = []
    
    L = len(layers)
    for l in range(L):
        A.append(np.zeros(layers[l]))
        A_derivate.append(np.zeros(layers[l]))
        error_outputs.append(np.zeros(layers[l]))
        
    for l in range(L-1):
        error_weights.append(np.zeros((layers[l],layers[l+1])))
        network_weights.append(np.random.rand(layers[l],layers[l+1]))

    return network_weights, error_weights, error_outputs, A, A_derivate 

In [6]:
class NeuralNetwork:
    # layers: información de la cantidad de neuronas por capa
    # activation: función de activación
    # type_loss: función de pérdida
    def __init__(self, layers, activation, type_loss):
        self.L = len(layers) # cantidad capas
        self.S = layers # cantidad de neuronas en cada capa
        self.activation = activation 
        self.type_loss = type_loss
        
        # asignar función gradiente
        if activation == sigmoid:
            self.activation_gradient = gradient_sigmoid
        if activation == relu:
            self.activation_gradient = gradient_relu
            

        # crear arquitectura
        self.weights, self. e_weights, self.e_output, self.neurons, self.n_gradient = create_matrices(layers)
        print("Red neuronal creada")
        print(" - Cantidad de capas   :", len(layers), layers)
        print(" - Activación neuronas :", activation.__name__)
        print(" - Función de pérdida  :", type_loss)

    
    def forward_pass(self, example):
        x = example[0].copy()
        y = example[1].copy()
        self.neurons[0] = x.copy()
        print("entrada", x)
        for l in range(self.L - 2): # por cada capa
            print("capa", l)
            for s in range(self.S[l+1]):
                print("-neurona",s)
                aux = np.dot(self.weights[l][:,s], self.neurons[l])
                self.neurons[l+1][s] = self.activation(aux).copy()
                self.n_gradient[l+1][s] = self.activation_gradient(aux).copy()
                print("soft", self.neurons[l+1][s])
                print("gradiente", self.n_gradient[l+1][s])
        print("pesos",self.weights)
        print("neuronas",self.neurons)
        print("capa",self.L-1)
        for s in range(self.S[self.L-1]): # capa final
            print("+capa", s)
            print("peso", self.weights[self.L-2][:,s])
            print("new",self.neurons[self.L-2])
            aux = np.dot(self.weights[self.L-2][:,s], self.neurons[self.L-2])
            self.neurons[self.L-1][s] = aux.copy()
            print(self.neurons[self.L-1][s])
        self.neurons[self.L-1] = softmax(self.neurons[self.L-1]).copy()    
        self.n_gradient[self.L-1] = gradient_softmax(self.neurons[self.L-1]).copy()
        print("soft", self.neurons[self.L-1])
        print("gradient soft", self.n_gradient[self.L-1])
        
        
    def backward_pass(self):
        pass
    
    def train(X, Y, epoch, learning_rate):
        pass
        
        
        
        

        
red = NeuralNetwork([2,3,1], activation=sigmoid, type_loss="cross")

red.forward_pass([[1,2],[3]])
print("neuronas", red.neurons)
print("gradiente", red.n_gradient)

Red neuronal creada
 - Cantidad de capas   : 3 [2, 3, 1]
 - Activación neuronas : sigmoid
 - Función de pérdida  : cross
entrada [1, 2]
capa 0
-neurona 0
soft 0.8589096182488509
gradiente 0.12118388592846409
-neurona 1
soft 0.623058525074124
gradiente 0.2348565994065812
-neurona 2
soft 0.8494122316737647
gradiente 0.1279110923567594
pesos [array([[0.5505776 , 0.25320507, 0.21197596],
       [0.6278427 , 0.12467273, 0.75901128]]), array([[0.64863308],
       [0.82776725],
       [0.66889304]])]
neuronas [[1, 2], array([0.85890962, 0.62305853, 0.84941223]), array([0.])]
capa 2
+capa 0
peso [0.64863308 0.82776725 0.66889304]
new [0.85890962 0.62305853 0.84941223]
1.6410305709273159
soft [1.]
gradient soft [0.]
neuronas [[1, 2], array([0.85890962, 0.62305853, 0.84941223]), array([1.])]
gradiente [array([0., 0.]), array([0.12118389, 0.2348566 , 0.12791109]), array([0.])]
